# SEP Tutorial Using astropy.io.fits (ASTR-19 Final Project)
This notebook recreates the SEP tutorial but uses **Astropy FITS** routines instead of fitsio.

### Goals:
- Load the FITS example image with `astropy.io.fits`
- Compute and subtract the background
- Detect astronomical sources using SEP
- Perform aperture photometry
- Create and save **four PNG figures**:
  1. Original image
  2. Background map
  3. Source detection
  4. Flux histogram

In [ ]:
import numpy as np
import sep
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline

Load the Example FITS Image And Background Subtraction

In [ ]:
data = fits.getdata("example.fits")
data = data.byteswap().newbyteorder()

# plot the image
plt.figure(figsize=(6,6))
plt.imshow(data, cmap = 'gray', origin = 'lower')
plt.colorbar(label='Pixel value')
plt.title('Original SEP Tutorial Image')
plt.savefig('fig1.png', dpi = 150, bbox_inches = 'tight')
plt.show()

In [ ]:
bkg = sep.Background(data)
bkg_image = bkg.back()
data_sub = data - bkg_image

# plot background
plt.figure(figsize=(6,6))
plt.imshow(bkg_image, cmap = 'gray', origin = 'lower')
plt.colorbar(label='Background level')
plt.title('Background Map')
plt.savefig('fig2.png', dpi = 150, bbox_inches = 'tight')
plt.show()

Source Extraction

In [ ]:
objects = sep.extract(data_sub, thresh = 1.5)
print("Number of sources detected is :", len(objects))
objects[:5]

Plot Detected Sources

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots(figsize = (6,6))
ax.imshow(data_sub, cmap = 'gray', origin = 'lower')
for obj in objects:
    e = Ellipse((obj['x'], obj['y']), 6*obj['a'], 6*obj['b'], obj['theta']*180/np.pi, edgecolor = 'blue', facecolor = 'none', linewidth = 0.8)
    ax.add_patch(e)
ax.set_title('Detected Sources')
plt.savefig('fig3.png', dpi = 150, bbox_inches = 'tight')
plt.show()

Aperture Photometry & Flux Histogram

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], r = 3.0)
plt.figure(figsize=(7,5))
plt.hist(flux, bins = 30, histtype = 'step')
plt.xlabel('Flux')
plt.ylabel('Number of sources')
plt.title('Flux Histogram')
plt.savefig('fig4.png', dpi = 150, bbox_inches = 'tight')
plt.show()